# Preliminaries

### Imports

In [1]:
import pandas as pd
import gzip

### Functions

In [2]:
def read_tsv_qz(file_path, zip=True, sep='\t'):
    if zip:
        with gzip.open(file_path, 'rt') as f:
            return pd.read_csv(f, delimiter=sep)
    else:
        return pd.read_csv(file_path, delimiter=',')

def write_tsv_qz(df, file_path, zip=True):
    if zip:
        with gzip.open(file_path, 'wt') as f:
            df.to_csv(f, sep='\t', index=False)
    else:
        df.to_csv(file_path, sep=',', index=False)
    
def convert_string_to_year(input_string):
    if len(input_string) == 4:
        return int(input_string)
    elif '/' in input_string:
        if '—' in input_string:
            return None
        
        aux = input_string.split('/')[2]
        if len(aux) == 4:
            return int(aux)
        else:
            return int('19' + aux)
    else:
        for word in input_string.split():
            if len(word) == 4 and word.isdigit():
                return int(word)
    return None

### Constants

In [3]:
title_akas_tsv = 'RawData/title.akas.tsv.gz'
title_basics_tsv = 'RawData/title.basics.tsv.gz'
title_ratings_tsv = 'RawData/title.ratings.tsv.gz'
title_principals_tsv = 'RawData/title.principals.tsv.gz'
title_crew_tsv = 'RawData/title.crew.tsv.gz'
books_csv = 'RawData/books_1.Best_Books_Ever.csv'
boxoffice_csv = 'RawData/boxoffice.csv'
data_api_csv = 'RawData/data_api.csv'
wikidata_csv = 'RawData/wikidata_query.csv'

join_column_title_akas = 'titleId'
join_column_title_basics = 'tconst'
join_column_title_ratings = 'tconst'
join_column_title_principals = 'tconst'

# Combining the movies and boxoffice datasets (plus wikidata and api data)

Reading the main tsv file:

In [4]:
df_title_basics = read_tsv_qz(title_basics_tsv)
df_title_basics = df_title_basics[df_title_basics['titleType'] == 'movie']
df_title_basics = df_title_basics.drop('endYear', axis=1)
df_title_basics = df_title_basics.drop('titleType', axis=1)
df_title_basics = df_title_basics[df_title_basics['startYear'] < '2024']
# remove rows that are not a movie,
# remove the endYear column as it is not relevant for movies (it is used for TV series)
# remove movies that are not released yet

/var/folders/td/hh6f_yhd28q6g56x8wwsht340000gn/T/ipykernel_11611/1708109223.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f, delimiter=sep)


Reading the boxoffice csv file

In [5]:
df_boxoffice = read_tsv_qz(boxoffice_csv, zip=False)
df_boxoffice = df_boxoffice.sort_values(by='lifetime_gross', ascending=False)
df_boxoffice = df_boxoffice.drop_duplicates(subset='title', keep='first')
df_boxoffice = df_boxoffice.dropna(subset=['studio'])
df_boxoffice = df_boxoffice.drop('rank', axis=1)

Combining the title_basics and boxofffice datasets based on title and year

In [6]:
merged_df = df_title_basics.merge(df_boxoffice, left_on='primaryTitle', right_on='title')
merged_df = merged_df.drop(columns=["title"])

Extensive filtering to ensure proper matching between movies

In [7]:
merged_df['startYear'] = merged_df['startYear'].astype(int)
merged_df = merged_df[(merged_df['startYear'] >= merged_df['year'] - 1) & (merged_df['startYear'] <= merged_df['year'] + 1)]
tconst_list = merged_df['tconst'].tolist()
merged_df = merged_df.drop(columns=["studio", "year"])
merged_df['lifetime_gross'] = merged_df['lifetime_gross'].astype(int)

Export movies not in our dataset

In [8]:
filtered_df = df_title_basics[~df_title_basics['tconst'].isin(tconst_list)]
filtered_df[['tconst']].to_csv('RawData/tconst_not_in_dataset_csv_filtered.csv', index=False)

In [9]:
del df_boxoffice
del filtered_df

Adding OMDb API boxoffice data

In [10]:
df_api = read_tsv_qz(data_api_csv, zip=False)
df_api = df_api.dropna(subset=['BoxOffice'])
df_api = df_api.drop_duplicates(subset=['tconst'], keep='first')
df_api = df_api.rename(columns={'BoxOffice': 'lifetime_gross'})
df_api['lifetime_gross'] = df_api['lifetime_gross'].astype(int)

Adding Wikidata query data

In [11]:
df_wikidata = read_tsv_qz(wikidata_csv, zip=False)
df_wikidata = df_wikidata.drop(columns=['item', 'itemLabel'])
df_wikidata = df_wikidata.rename(columns={'domesticGross': 'lifetime_gross', 'imdbID': 'tconst'})
df_wikidata = df_wikidata.drop_duplicates(subset=['tconst'], keep='first')

Concatenate wikidata and api datasets

In [12]:
df_wikidata_api = pd.concat([df_wikidata, df_api])
df_wikidata_api = df_wikidata_api.drop_duplicates(subset=['tconst'], keep='first')
df_wikidata_api = df_wikidata_api.rename(columns={'tconst': 'titleId'})

In [13]:
del df_wikidata
del df_api

Merge again

In [14]:
aux = df_title_basics.merge(df_wikidata_api, left_on='tconst', right_on='titleId')
aux = aux.drop(columns=["titleId"])

In [15]:
merged_df = pd.concat([merged_df, aux])
merged_df = merged_df.drop_duplicates(subset=['tconst'], keep='first')
tconst_list = merged_df['tconst'].tolist()

In [16]:
del aux
del df_title_basics
del df_wikidata_api

Reading the akas tsv file and extracting new features

In [17]:
df_akas = read_tsv_qz(title_akas_tsv)
df_akas = df_akas[df_akas['titleId'].isin(tconst_list)]
df_akas = df_akas.sort_values(by='ordering', ascending=False)
df_akas = df_akas.drop_duplicates(subset='titleId', keep='first')
df_akas = df_akas.drop(columns=['title', 'region', 'language', 'types',
       'attributes', 'isOriginalTitle'])
df_akas = df_akas.rename(columns={'ordering': 'nrOfReleases'})

In [18]:
merged_df = merged_df.merge(df_akas, left_on='tconst', right_on='titleId')
merged_df = merged_df.drop(columns=["titleId"])

tconst_list = merged_df['tconst'].tolist()

In [19]:
del df_akas

Reading the crew tsv file and extracting new features

In [20]:
df_crew = read_tsv_qz(title_crew_tsv)
df_crew = df_crew[df_crew['tconst'].isin(tconst_list)]

In [21]:
df_crew['directors'] = df_crew['directors'].str.split(',')  # Split the directors column by comma
df_crew = df_crew.explode('directors')  # Explode the rows with multiple directors

In [22]:
df_crew_full = read_tsv_qz(title_crew_tsv)
df_crew_full['directors'] = df_crew_full['directors'].str.split(',')
df_crew_full = df_crew_full.explode('directors')
director_counts = df_crew_full['directors'].value_counts()

In [23]:
del df_crew_full

In [24]:
df_crew['director_count'] = df_crew['directors'].map(director_counts)

In [25]:
del director_counts

In [26]:
df_crew = df_crew.sort_values(by='director_count', ascending=False)
df_crew = df_crew.drop_duplicates(subset='tconst', keep='first')
df_crew = df_crew.drop(columns=['writers'])
df_crew = df_crew.rename(columns={'tconst': 'titleId'})

In [27]:
merged_df = merged_df.merge(df_crew, left_on='tconst', right_on='titleId')
merged_df = merged_df.drop(columns=["titleId"])
merged_df = merged_df[merged_df['directors'] != '\\N']
merged_df = merged_df.drop(columns=['directors'])
merged_df = merged_df[merged_df['genres'] != '\\N']
merged_df = merged_df[merged_df['runtimeMinutes'] != '\\N']

tconst_list = merged_df['tconst'].tolist()

In [28]:
del df_crew

Reading the principals tsv file

In [29]:
df_principal = read_tsv_qz(title_principals_tsv)
df_principal = df_principal[df_principal['tconst'].isin(tconst_list)]
df_principal['max_ordering'] = df_principal.groupby('tconst')['ordering'].transform('max')
df_principal.loc[df_principal['category'].isin(['actor', 'actress']), 'category'] = 'a'
df_principal['count_category_a'] = df_principal.groupby('tconst')['category'].transform(lambda x: x[x == 'a'].count())
df_principal = df_principal.drop(columns=['ordering', 'nconst', 'job', 'characters', 'category'])
df_principal = df_principal.drop_duplicates(subset='tconst', keep='first')
df_principal = df_principal.rename(columns={'tconst': 'titleId'})

In [30]:
merged_df = merged_df.merge(df_principal, left_on='tconst', right_on='titleId')
merged_df = merged_df.drop(columns=["titleId"])
merged_df = merged_df.drop(columns=['originalTitle'])
merged_df = merged_df.rename(columns={'startYear': 'releaseYear', 'count_category_a': 'nrOfActors', 'lifetime_gross': 'lifetimeGross', 'director_count': 'director_nrOfMovies', 'max_ordering': 'nrOfEmployees'})

tconst_list = merged_df['tconst'].tolist()

In [31]:
del df_principal

In [32]:
df_ratings = read_tsv_qz(title_ratings_tsv)
df_ratings = df_ratings[df_ratings['tconst'].isin(tconst_list)]
df_ratings = df_ratings.rename(columns={'tconst': 'titleId'})

In [33]:
merged_df = merged_df.merge(df_ratings, left_on='tconst', right_on='titleId')

tconst_list = merged_df['tconst'].tolist()
merged_df = merged_df.drop(columns=["titleId", "tconst", "primaryTitle"])

In [34]:
del df_ratings

In [39]:
aux_df = read_tsv_qz(title_basics_tsv)

/var/folders/td/hh6f_yhd28q6g56x8wwsht340000gn/T/ipykernel_11611/1708109223.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f, delimiter=sep)


In [38]:
aux_df = read_tsv_qz(title_basics_tsv)
aux_df = aux_df[aux_df['titleType'] == 'movie']
aux_df = aux_df[['tconst', 'startYear']]
aux_df = aux_df[aux_df['startYear'] != '\\N']
aux_df['startYear'] = aux_df['startYear'].astype(int)
release_year_counts = aux_df['startYear'].value_counts()

/var/folders/td/hh6f_yhd28q6g56x8wwsht340000gn/T/ipykernel_11611/1708109223.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f, delimiter=sep)


In [41]:
merged_df['releaseYear'] = merged_df['releaseYear'].astype(int)

In [42]:
del aux_df

In [43]:
merged_df['sameYearTotalMoviesReleased'] = merged_df['releaseYear'].map(release_year_counts)

Export the merged dataset

In [44]:
write_tsv_qz(merged_df, 'CleanedData/dataset.csv', zip=False)